# SANS2D data reduction

In [ ]:
import scipp as sc
from ess import loki, sans
import scippneutron as scn
# from ess.loki.load_sans2d import load_isis, load_rkh_q, load_rkh_wav#, load_and_apply_masks, apply_tof_mask
# from ess.loki.sans2d.transform_coordinates import setup_offsets, setup_geometry
# from ess import sans
# from scippneutron.tof.conversions import beamline, elastic_Q

## Loading data files

In [ ]:
path = 'SANS2D_data'

sample_run_number = 63114
background_run_number = 63159
direct_run_number = 63091

In [ ]:
ds = sc.Dataset()

#Using only one-forth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

# Make common tof bins so that dataset items are aligned
tof_bins = sc.linspace(dim='tof', start=0, stop=100000, num=2, unit='us')

# Sample measurement
ds['sample'] = loki.io.load_sans2d(filename=f'{path}/SANS2D000{sample_run_number}.nxs',
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)
# Background is a measurement with the solvent which the sample is placed in
ds['background'] = loki.io.load_sans2d(filename=f'{path}/SANS2D000{background_run_number}.nxs',
                                       spectrum_size=spectrum_size, tof_bins=tof_bins)
# Direct measurement is with the empty sample holder/cuvette
ds['direct'] = loki.io.load_sans2d(filename=f'{path}/SANS2D000{direct_run_number}.nxs',
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

In [ ]:
ds

## Apply offsets to pixel positions

In [ ]:
# Solid angle values
pixel_size = 0.0035 * sc.units.m
pixel_length = 0.002033984375 * sc.units.m

# Coordinate trasnformation
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m
# z_offset = 0.0 * sc.units.m

In [ ]:
# Change sample position
ds.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
ds.coords["position"].fields.y += bench_pos_y_offset

for key in ds:
    ds[key].attrs["monitor4"].value.coords["position"].fields.z += monitor4_pos_z_offset

# Now shift pixels positions to get the correct beam center
ds.coords['position'].fields.x += x_offset
ds.coords['position'].fields.y += y_offset

In [ ]:
# scn.instrument_view(ds['sample'], pixel_size=0.0075)

## Mask bad pixels

In [ ]:
mask_edges = (
    (sc.abs(ds.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
    (sc.abs(ds.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    ds.coords['position'].fields.x**2 +
    ds.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')
# cylindrical_radius = sc.sqrt(
#     ds.coords['position'].fields.x**2 +
#     ds.coords['position'].fields.y**2)
# mask_center = (cylindrical_radius < sc.scalar(0.04, unit='m')) | (cylindrical_radius > sc.scalar(0.4, unit='m'))

for key in ds:
    ds[key].masks['edges'] = mask_edges
    ds[key].masks['center'] = mask_center

In [ ]:
scn.instrument_view(ds['sample'], pixel_size=0.0075)

## Convert data to wavelength

In [ ]:
graph = sans.conversions.sans_elastic()
sc.show_graph(graph, simplified=True)

In [ ]:
from scipp.constants import g
ds.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g
ds_wav = ds.transform_coords("wavelength", graph=graph)
ds_wav

In [ ]:
graph_monitor = sans.conversions.sans_monitor()
for key in ds:
    for m in ['monitor2', 'monitor4']:
        ds_wav[key].attrs[m].value = ds[key].attrs[m].value.transform_coords("wavelength", graph=graph_monitor)
ds_wav

## Compute normalization term

### Detector efficiency

In [ ]:
from ess.wfm.tools import to_bin_edges

# Load efficency correction for main detector
detector_efficiency_file = 'DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'
detector_efficiency = loki.io.load_rkh_wav(filename=f'{path}/{detector_efficiency_file}')

detector_efficiency.coords['wavelength'] = to_bin_edges(detector_efficiency.coords['wavelength'], 'wavelength')
wavelength_min = sc.scalar(2.0, unit='angstrom')
wavelength_max = sc.scalar(16.0, unit='angstrom')
detector_efficiency = detector_efficiency['wavelength', wavelength_min:wavelength_max].copy(deep=True)

In [ ]:
sc.plot(detector_efficiency)

### Subtract background from monitors and align them to the same wavelength range

In [ ]:
wavelength_bins = detector_efficiency.coords['wavelength']
threshold = 30.0 * sc.units.counts

sample_incident = sans.normalization.subtract_background_and_rebin(
    ds['sample'].attrs["monitor2"].value, wavelength_bins, threshold)
sample_transmission = sans.normalization.subtract_background_and_rebin(
    ds['sample'].attrs["monitor4"].value, wavelength_bins, threshold)
direct_incident = sans.normalization.subtract_background_and_rebin(
    ds['direct'].attrs["monitor2"].value, wavelength_bins, threshold)
direct_transmission = sans.normalization.subtract_background_and_rebin(
    ds['direct'].attrs["monitor4"].value, wavelength_bins, threshold)

### Transmission fraction

In [ ]:
transmission_fraction = sans.normalization.transmission_fraction(
    sample_incident=sample_incident, sample_transmission=sample_transmission,
    direct_incident=direct_incident, direct_transmission=direct_transmission)
transmission_fraction

In [ ]:
transmission_fraction.plot()

### Solid Angle

In [ ]:
solid_angle = sans.normalization.solid_angle(ds['sample'], pixel_size=pixel_size, pixel_length=pixel_length)
solid_angle

### The denominator term

In [ ]:
denominator = solid_angle * (sample_incident * transmission_fraction) * detector_efficiency.data
denominator.coords['position'] = ds['sample'].coords['position']
denominator.coords['gravity'] = ds['sample'].coords['gravity']
denominator

## Convert to Q

In [ ]:
ds_q = ds_wav.transform_coords("Q", graph=graph)
ds_q

In [ ]:
q_edges = sc.array(dims=['Q'], values=[0.01, 0.6], unit='1/angstrom')
sample_q_binned = sc.bin(ds_q['sample'], edges=[q_edges])
sample_q_binned

In [ ]:
sample_q_summed = sample_q_binned.bins.concat('spectrum')

In [ ]:
sc.plot(sample_q_summed["wavelength", 0], norm="log")

### Convert denominator to Q

In [ ]:
denominator_q = denominator.transform_coords("Q", graph=graph)
denominator_q

In [ ]:
den_q_summed = denominator_q.sum('spectrum')
den_q_summed.plot()

## Normalize the sample

In [ ]:
sample_normalized = sample_q_summed.bins / sc.lookup(func=den_q_summed, dim='wavelength')
sample_normalized

In [ ]:
sc.plot(sample_normalized['wavelength', 0], norm='log')

### Process the background run

In [ ]:
transmission_fraction_background = sansnorm.transmission_fraction(
    sample=ds['background'],
    direct=ds['direct'],
    wavelength_bins=wavelength_bins,
    threshold=threshold)
transmission_fraction_background.plot()

In [ ]:
monitor_background = sansnorm.substract_background_and_rebin(
    ds['background'].attrs['monitor2'].value, wavelength_bins, threshold).copy(deep=True)
norm_background = (monitor_background * transmission_fraction_background) * efficiency_cropped.data
denominator_background = solid_angle * norm_background
denominator_background.coords['position'] = ds.coords['position']
denominator_background.coords['gravity'] = ds.coords['gravity']
denominator_background

In [ ]:
q_edges = sc.array(dims=['Q'], values=[0.01, 0.6], unit='1/angstrom')
background_q_binned = sc.bin(ds_q['background'], edges=[q_edges])
background_q_binned

In [ ]:
background_q_summed = background_q_binned.bins.concat('spectrum')

In [ ]:
denominator_background_q = denominator_background.transform_coords("Q", graph=graph)
denominator_background_q

In [ ]:
den_back_q_summed = denominator_background_q.sum('spectrum')
den_back_q_summed.plot()

In [ ]:
background_normalized = background_q_summed.bins / sc.lookup(func=den_back_q_summed, dim='wavelength')
background_normalized

In [ ]:
sc.plot(background_normalized['wavelength', 0], norm='log')

## Subtract background

In [ ]:
q_edges = sc.linspace(dim='Q', start=0.01, stop=0.6, num=201, unit='1/angstrom')
reduced = sc.bin(sample_normalized['wavelength', 0], edges=[q_edges]).bins.sum() - sc.bin(
    background_normalized['wavelength', 0], edges=[q_edges]).bins.sum()

In [ ]:
reduced

In [ ]:
reduced.plot()

In [ ]:
reduced.plot(norm='log')

## Wavelength slices

In [ ]:
wav_edges = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=11, unit='angstrom')

In [ ]:
sample_q_summed

In [ ]:
sample_q_summed.bins.coords['wavelength'] = sample_q_summed.bins.attrs.pop('wavelength')

In [ ]:
sample_slices = sc.bin(sample_q_summed, edges=[wav_edges])

In [ ]:
sample_slices

In [ ]:
sc.plot(sc.collapse(sample_slices, keep='Q'), norm='log')

### Normalize

In [ ]:
sample_slices_normalized = sample_slices.bins / sc.lookup(func=den_q_summed, dim='wavelength')
sample_slices_normalized

In [ ]:
sc.plot(sc.collapse(sample_slices_normalized, keep='Q'), norm='log')

### Subtract background

In [ ]:
background_q_summed.bins.coords['wavelength'] = background_q_summed.bins.attrs.pop('wavelength')
background_slices = sc.bin(background_q_summed, edges=[wav_edges])
background_slices_normalized = background_slices.bins / sc.lookup(func=den_back_q_summed, dim='wavelength')

In [ ]:
q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=201, unit='1/angstrom')
reduced_slices = sc.histogram(sample_slices_normalized, bins=q_bins) - sc.histogram(
    background_slices_normalized, bins=q_bins)
reduced_slices

In [ ]:
sc.plot(sc.collapse(reduced_slices, keep='Q'), norm='log')

In [ ]:
sc.plot(sc.collapse(reduced_slices, keep='Q'))

In [ ]:
p = sc.plot(sc.collapse(reduced_slices, keep='Q'), norm='log', scale={'Q': 'log'})
p.ax.set_xlim(0.01, 0.6)
p

## Wavelength slices take 2

In [ ]:
ds_q

In [ ]:
sample_slices = sc.bin(ds_wav['sample'], edges=[wav_edges])
sample_slices

In [ ]:
q_slices = sample_slices.transform_coords("Q", graph=graph)
# q_slices = q_slices.rename_dims({"wavelength": "Q"})
q_slices

In [ ]:
q_slices_summed = q_slices.bins.concat('spectrum')
q_slices_summed